In [19]:
import csv
import pandas as pd

data = pd.read_csv('ground_truth.csv')
data[:20]

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,retro
0,0.591,0.764,1,-5.484,0.0483,0.03830,0.000000,0.1030,0.478,169.928,4,False
1,0.741,0.691,10,-7.395,0.0672,0.02210,0.000000,0.0476,0.892,150.087,4,False
2,0.761,0.525,11,-6.900,0.0944,0.44000,0.000007,0.0921,0.531,80.870,4,False
3,0.762,0.766,7,-3.955,0.0343,0.00776,0.000070,0.1280,0.442,130.001,4,False
4,0.808,0.897,11,-3.712,0.0348,0.04690,0.000031,0.3640,0.591,126.026,4,False
5,0.588,0.701,9,-6.390,0.0402,0.00813,0.000000,0.2000,0.443,104.988,4,False
6,0.824,0.764,5,-4.175,0.0854,0.08880,0.002940,0.1170,0.881,107.998,4,False
7,0.788,0.859,2,-2.724,0.0856,0.28100,0.000000,0.0424,0.822,141.020,4,False
8,0.727,0.739,8,-3.805,0.0411,0.36500,0.001370,0.1610,0.533,93.005,4,False
9,0.737,0.846,1,-4.510,0.2200,0.00614,0.000000,0.0486,0.546,87.981,4,False


In [ ]:
import sagemaker

prefix = 'sagemaker/tutorial-autopilot/input'
sess   = sagemaker.Session()

uri = sess.upload_data(path="ground_truth.csv", key_prefix=prefix)
print(uri)

In [68]:
import boto3
import json
from json.decoder import JSONDecodeError
import requests
import sys 
import pandas as pd
import csv

song_counter =0 

CLIENT_ID = "83ae548c8bdb40a9a65a4b6e2a174bd6"
CLIENT_SECRET = "9961fe6a1fe94d11a256d13a65b44efb"


AUTH_URL = 'https://accounts.spotify.com/api/token'
base_url = "https://api.spotify.com/v1/"

playlist = "5PKvOAuZ9NNyKyhrHeIDlF"

ep_name = 'retro-autopilot-best-model'
sm_rt = boto3.Session().client('runtime.sagemaker')

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

final_attributes = []

auth_response_data = auth_response.json()
access_token = auth_response_data["access_token"]

playlist = requests.get(base_url+"playlists/"+playlist, headers=headers)
spotify_playlist_output = playlist.json()

items = spotify_playlist_output["tracks"]["items"]
items_array = []
print(len(items))
for item in items:
    id = (item["track"]["id"])
    items_array.append(id)

    song_attr = []

for array_item in items_array:
    song_attributes = requests.get(base_url+"audio-features/"+array_item, headers=headers)
    json_attr = song_attributes.json()
    is_retro = True
    song = [
        json_attr["danceability"],
        json_attr["energy"],
        json_attr["key"],
        json_attr["loudness"],
        json_attr["speechiness"],
        json_attr["acousticness"],
        json_attr["instrumentalness"],
        json_attr["liveness"],
        json_attr["valence"],
        json_attr["tempo"],
        json_attr["time_signature"],
        is_retro
    ]

    final_attributes.append(song)
    print("finished song" + str(song_counter))
    song_counter += 1

print(json.dumps(final_attributes))
with open("songs.csv", 'w') as csvfile: 
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","time_signature","retro"])
    for song in final_attributes:
        csvwriter.writerow(song) 
    csvfile.close() 

with open('songs.csv') as csv_in:
    
    
    lines = csv_in.readlines()
    for l in lines[1:2000]:   # Skip header
        l = l.split(',')      # Split CSV line into features
        label = l[-1]         # Store 'yes'/'no' label
        l = l[:-1]            # Remove label
        l = ','.join(l)
        print(type(l))
        response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv',Accept='text/csv', Body=l)
        response = response['Body'].read().decode("utf-8")
        print(response)
        print(l)

        
    

65
finished song0
finished song1
finished song2
finished song3
finished song4
finished song5
finished song6
finished song7
finished song8
finished song9
finished song10
finished song11
finished song12
finished song13
finished song14
finished song15
finished song16
finished song17
finished song18
finished song19
finished song20
finished song21
finished song22
finished song23
finished song24
finished song25
finished song26
finished song27
finished song28
finished song29
finished song30
finished song31
finished song32
finished song33
finished song34
finished song35
finished song36
finished song37
finished song38
finished song39
finished song40
finished song41
finished song42
finished song43
finished song44
finished song45
finished song46
finished song47
finished song48
finished song49
finished song50
finished song51
finished song52
finished song53
finished song54
finished song55
finished song56
finished song57
finished song58
finished song59
finished song60
finished song61
finished song62